# Initial Baseline

## Initial Setup

In [1]:
!pip install -q sentencepiece

In [2]:
!pip install -q transformers

In [3]:
!pip install -q datasets

In [4]:
!pip install -q evaluate

In [5]:
!pip install -q rouge_score

In [6]:
from datasets import load_dataset

In [7]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Data 266 Project/sample_data.csv', split='train')

In [9]:
dataset

Dataset({
    features: ['subreddit', 'id', 'content', 'summary'],
    num_rows: 36000
})

In [10]:
dataset = dataset.train_test_split(test_size = 0.1)

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['subreddit', 'id', 'content', 'summary'],
        num_rows: 32400
    })
    test: Dataset({
        features: ['subreddit', 'id', 'content', 'summary'],
        num_rows: 3600
    })
})

In [12]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration, AutoConfig

pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
pmodel.summary()

Model: "tf_pegasus_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  569748480 
                                                                 
 final_logits_bias (BiasLay  multiple                  96103     
 er)                                                             
                                                                 
Total params: 569844583 (2.12 GB)
Trainable params: 569748480 (2.12 GB)
Non-trainable params: 96103 (375.40 KB)
_________________________________________________________________


In [14]:
config = AutoConfig.from_pretrained("google/pegasus-xsum")

config

PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,


In [15]:
prefix = "summarize: "

body_name = "content"
label_name = "summary"

def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples[body_name]]
  model_inputs = ptokenizer(inputs, max_length=512, truncation=True)
  labels = ptokenizer(text_target=examples[label_name], max_length=256, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [16]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/32400 [00:00<?, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=ptokenizer, model=pmodel, return_tensors="tf")

In [18]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [19]:
tf_train_set = pmodel.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

tf_test_set = pmodel.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

In [20]:
import tensorflow as tf

pmodel.compile(optimizer=optimizer)

In [21]:
import evaluate

rouge = evaluate.load("rouge")

In [22]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = ptokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, ptokenizer.pad_token_id)
    decoded_labels = ptokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != ptokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [23]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [24]:
del dataset

In [25]:
pmodel.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1, callbacks=[metric_callback])

ResourceExhaustedError: ignored